# Generating Rest Frame SEDs from a Parametric Galaxy including dust emission
<br>
Example for generating the rest-frame spectrum for a parametric galaxy including dust emission


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from synthesizer.filters import FilterCollection
from synthesizer.grid import Grid
from synthesizer.parametric.sfzh import SFH, ZH, generate_sfzh
from synthesizer.parametric.galaxy import Galaxy
from synthesizer.plt import single, single_histxy, mlabel
from synthesizer.dust import Greybody, Casey12
from unyt import yr, Myr, K, um
from astropy.cosmology import Planck18 as cosmo

In [ ]:
# Define the grid
grid_name = "test_grid"
grid_dir = "../../../tests/test_grid/"
grid = Grid(grid_name, grid_dir=grid_dir)

In [ ]:
# define the parameters of the star formation and metal enrichment histories
sfh_p = {'duration': 10 * Myr}
Z_p = {'log10Z': -2.0}  # can also use linear metallicity e.g. {'Z': 0.01}
stellar_mass = 1E8

# define the functional form of the star formation and metal enrichment histories
sfh = SFH.Constant(sfh_p)  # constant star formation
Zh = ZH.deltaConstant(Z_p)  # constant metallicity

# get the 2D star formation and metal enrichment history for the given SPS grid. This is (age, Z).
sfzh = generate_sfzh(grid.log10age, grid.metallicity, sfh, Zh, stellar_mass=stellar_mass)

# create a galaxy object
galaxy = Galaxy(sfzh)

In [ ]:
# # --- simple dust and gas screen
galaxy.get_spectra_screen(grid, tauV = 0.3, fesc = 0.5)
print("Simple dust and gas screen")
galaxy.plot_spectra()


In [ ]:
print(galaxy.spectra['intrinsic'].get_bolometric_luminosity())
print(galaxy.spectra['attenuated'].get_bolometric_luminosity())

In [ ]:
emissionmodel = Greybody(50*K, 1.5)

In [ ]:
galaxy.get_spectra_dust(emissionmodel)


plt.loglog(galaxy.spectra['intrinsic'].lam, galaxy.spectra['stellar'].nu*galaxy.spectra['intrinsic'].lnu, label = 'intrinsic')
plt.loglog(galaxy.spectra['dust'].lam, galaxy.spectra['stellar'].nu*galaxy.spectra['dust'].lnu, label = 'dust emission')
plt.loglog(galaxy.spectra['total'].lam, galaxy.spectra['stellar'].nu*galaxy.spectra['total'].lnu, label = 'total')
plt.xlabel(r'$\lambda$/$\AA$')
plt.ylabel(r'$\nu$L$_{\nu}$(erg/s)')
plt.ylim([1E41, 1E46])
plt.xlim([1E3, 1E7])
plt.legend()

In [ ]:
emissionmodel = Casey12(T=50*K, emissivity=1.5, alpha=2., lam_0=100*um)

In [ ]:
galaxy.get_spectra_dust(emissionmodel)

plt.loglog(galaxy.spectra['intrinsic'].lam, galaxy.spectra['stellar'].nu*galaxy.spectra['intrinsic'].lnu, label = 'intrinsic')
plt.loglog(galaxy.spectra['dust'].lam, galaxy.spectra['stellar'].nu*galaxy.spectra['dust'].lnu, label = 'dust emission')
plt.loglog(galaxy.spectra['total'].lam, galaxy.spectra['stellar'].nu*galaxy.spectra['total'].lnu, label = 'total')
plt.xlabel(r'$\lambda$/$\AA$')
plt.ylabel(r'$\nu$L$_{\nu}$(erg/s)')
plt.ylim([1E41, 1E46])
plt.xlim([1E3, 1E7])
plt.legend()